In [1]:
import os
import ants
import numpy as np
import pandas as pd
import tensorflow as tf

2022-12-02 00:11:07.421866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 00:11:07.753891: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 00:11:08.961833: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-02 00:11:08.962031: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [3]:
img = ants.image_read("I31205.nii")
img

ValueError: File I31205.nii does not exist!

In [4]:
ants.plot(img)

NameError: name 'img' is not defined

In [ ]:
! antsRegistrationSyNQuick.sh -d 3 -f data/test/I30968.nii -m data/tpl-MNI305_T1w.nii.gz -o output/

In [ ]:
ants.plot(ants.image_read("output/InverseWarped.nii.gz"))

In [ ]:
! antsBrainExtraction.sh -d 3\
    -a output/InverseWarped.nii.gz\
        -e data/tpl-MNI305_desc-head_mask.nii.gz\
            -m data/tpl-MNI305_desc-brain_mask.nii.gz\
                -o output/ 

In [ ]:
ants.image_read("data/output/CN_I30968_BrainExtractionBrain.nii.gz")

In [ ]:
ants.image_read("data/output/MCI_I31392_BrainExtractionBrain.nii.gz")

In [ ]:
ants.plot(ants.image_read("data/output/CN_I30968_BrainExtractionBrain.nii.gz"))

In [ ]:
ants.plot(ants.image_read("data/output/MCI_I31392_BrainExtractionBrain.nii.gz"))

In [ ]:
description = pd.read_csv('data/test_data.csv')
description.head()

In [ ]:
def preprocess_nii(filename, path):
    
    test_db = 'data/test'
    
    REG_DB_SUBFOLDERS = ['AD/', 'MCI/', 'CN/']
    REG_Folder = 'data/registered_data/'
    Output_Folder = 'data/output/'
    
    moving = "data/tpl-MNI305_T1w.nii.gz"
    head_mask = "data/tpl-MNI305_desc-head_mask.nii.gz"
    brain_mask = "data/tpl-MNI305_desc-brain_mask.nii.gz"
    
    image_ID = filename[:-4]
    row_index = description.index[description['Image Data ID'] == image_ID].tolist()[0]
    row = description.iloc[row_index]
    label = row['Group']
    
    registration_file = os.path.join(REG_Folder, label, image_ID)
    output_file = os.path.join(Output_Folder)
    
    fixed = f"{path}/{filename}"
    ! antsRegistrationSyNQuick.sh -d 3 -f {fixed} -m {moving} -o {registration_file}_ -p f -n 4
    ! antsBrainExtraction.sh -d 3 -a {registration_file}_InverseWarped.nii.gz -e {head_mask} -m {brain_mask} -o {output_file}{label}_{image_ID}_ 
    

In [ ]:
test_db = 'data/test'

for nii in os.listdir(test_db):
    try:
        preprocess_nii(nii, test_db)
    except RuntimeError:
        print('Exception with', os.path.join(nii))

In [ ]:
''' 
  Create a TFRecord file, including the information
      of the specified images
      
      Parameters:
        img_filenames -- Array with the path to every
                         image that is going to be included
                         in the TFRecords file.
        tf_rec_filename -- Name of the TFRecords file.
'''

In [ ]:
output_folder = "data/output"
for nii in os.listdir(output_folder):
        if nii.endswith("_BrainExtractionBrain.nii.gz"):
            image_ID = nii.split("_")[1]
            print(image_ID)

In [15]:
DB_TF_3D_PATH = 'data/output'

TFREC_3D_SS_TRAIN = 'train.3D.skull_stripped.tfrecords'
TFREC_3D_SS_TEST = 'test.3D.skull_stripped.tfrecords'
TFREC_3D_SS_VAL = 'validation.3D.skull_stripped.tfrecords'

train_tfrec = os.path.join(DB_TF_3D_PATH, TFREC_3D_SS_TRAIN)
test_tfrec = os.path.join(DB_TF_3D_PATH, TFREC_3D_SS_TEST)
val_tfrec = os.path.join(DB_TF_3D_PATH, TFREC_3D_SS_VAL)

In [ ]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))
  
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def create_tf_record(tf_rec_filename):
    LABELS = {'CN': 0, 'MCI': 1, 'AD': 2}
    writer = tf.io.TFRecordWriter(tf_rec_filename)
    output_folder = "data/output"
    for nii in os.listdir(output_folder):
        if nii.endswith("_BrainExtractionBrain.nii.gz"):
            img = ants.image_read(os.path.join(output_folder, nii)).numpy()
            label = LABELS[nii.split("_")[0]]
            image_ID = nii.split("_")[1]
            age_and_sex = description.loc[description['Image Data ID'] == image_ID, ['Age', 'Sex']].iloc[0]
            feature = {'label': _int64_feature(label),
               'subject_age': _int64_feature(age_and_sex[0]),
               'subject_sex': _bytes_feature(age_and_sex[1].encode('utf-8')),
               'image_id': _bytes_feature(image_ID.encode('utf-8')),
               'image': _float_feature(img.ravel())}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
        else:
            pass
    writer.close()

In [ ]:
create_tf_record(train_tfrec)

In [1]:
from antspynet.architectures import create_resnet_model_3d

model = create_resnet_model_3d((172, 220, 156, 1))
model.summary()

2022-11-25 14:52:03.152749: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 14:52:03.479369: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-25 14:52:04.537388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-25 14:52:04.537536: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 172, 220, 1  0           []                               
                                56, 1)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 86, 110, 78,  22016       ['input_1[0][0]']                
                                 64)                                                              
                                                                                                  
 batch_normalization (BatchNorm  (None, 86, 110, 78,  256        ['conv3d[0][0]']                 
 alization)                      64)                                                          

In [4]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['acc'])

In [6]:
def _decode(serialized_example):
    IMG_SHAPE = (172, 220, 156, 1)
    feature = tf.parse_single_example(
        serialized_example,
        features = {
            'image': tf.FixedLenFeature(IMG_SHAPE, tf.float32),
            'label': tf.FixedLenFeature([], tf.int64),
            'subject_sex': tf.FixedLenFeature([], tf.string),
            'subject_age': tf.FixedLenFeature([], tf.int64),
            'image_id': tf.FixedLenFeature([], tf.string)
        }
    )
    
    return feature['image'], feature['label']

In [7]:
def dataset_parser(filepath, batch_size, n_classes=3):
    
    dataset = tf.data.TFRecordDataset(filepath).map(_decode)
    
    dataset = dataset.shuffle(SHUFFLE_BUFFER).repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)
    
    iterator = dataset.make_one_shot_iterator()
    image, label = iterator.get_next()
    label = tf.one_hot(label, n_classes)
    
    return image, label

In [18]:
image_tensor, label_tensor = dataset_parser(train_tfrec, 2)

AttributeError: in user code:

    File "/tmp/ipykernel_27579/1931094159.py", line 3, in _decode  *
        feature = tf.parse_single_example(

    AttributeError: module 'tensorflow' has no attribute 'parse_single_example'


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(x=image_tensor, y=label_tensor, 
                    epochs=20,
                    callbacks=[es])

In [ ]:
create_tf_record(train_tfrec)
create_tf_record(test_set, test_tfrec)
create_tf_record(validation_set, val_tfrec)

In [ ]:
def slices_matrix_2D(img):
  ''' Transform a 3D MRI image into a 2D image, by obtaining 9 slices 
      and placing them in a 4x4 two-dimensional grid.
      
      All 16 cuts are from a horizontal/axial view. They are selected
      from the 30th to the 60th level of the original 3D image.
      
      Parameters:
        img -- np.ndarray with the 3D image
        
      Returns:
        np.ndarray -- The resulting 2D image
  '''
  
  # create the final 2D image 
  image_2D = np.empty(IMG_2D_SHAPE)
  
  # set the limits and the step
  TOP = 60
  BOTTOM = 30
  STEP = 2
  N_CUTS = 16
  
  # iterator for the cuts
  cut_it = TOP
  # iterator for the rows of the 2D final image
  row_it = 0
  # iterator for the columns of the 2D final image
  col_it = 0
  
  for cutting_time in range(N_CUTS):
    
    # cut
    cut = img[cut_it, :, :]
    cut_it -= STEP
    
    # reset the row iterator and move the
    # col iterator when needed
    if cutting_time in [4, 8, 12]:
      row_it = 0
      col_it += cut.shape[1]
    
    # copy the cut to the 2D image
    for i in range(cut.shape[0]):
      for j in range(cut.shape[1]):
        image_2D[i + row_it, j + col_it] = cut[i, j]
    row_it += cut.shape[0]
  
  # return the final 2D image, with 3 channels
  # this is necessary for working with most pre-trained nets
  return np.repeat(image_2D[None, ...], 3, axis=0).T
  #return image_2D

In [ ]:
def load_image_2D(abs_path, labels):
  ''' Load an image (.nii) and its label, from its absolute path.
      Transform it into a 2D image, by obtaining 16 slices and placing them
      in a 4x4 two-dimensional grid.
      
      Parameters:
        abs_path -- Absolute path, filename included
        labels -- Label mapper
        
      Returns:
        img -- The .nii image, converted into a numpy array
        label -- The label of the image (from argument 'labels')
        
  '''
  
  # obtain the label from the path (it is the last directory name)
  label = labels[abs_path.split('/')[-2]]
  
  # load the image with SimpleITK
  sitk_image = sitk.ReadImage(abs_path)
  
  # transform into a numpy array
  img = sitk.GetArrayFromImage(sitk_image)
  
  # apply whitening
  img = preprocessing.whitening(img)
  
  # make the 2D image
  img = slices_matrix_2D(img)
  
  return img, label

In [ ]:
train_tfrec2D = os.path.join(DB_TF_2D_PATH, TFREC_2D_SS_TRAIN)
test_tfrec2D = os.path.join(DB_TF_2D_PATH, TFREC_2D_SS_TEST)
val_tfrec2D = os.path.join(DB_TF_2D_PATH, TFREC_2D_SS_VAL)

In [ ]:
def create_tf_record_2D(img_filenames, tf_rec_filename, labels):
  ''' Create a TFRecord file, including the information
      of the specified images, after converting them into 
      a 2D grid.
      
      Parameters:
        img_filenames -- Array with the path to every
                         image that is going to be included
                         in the TFRecords file.
        tf_rec_filename -- Name of the TFRecords file.
        labels -- Label mapper
  '''
  
  # open the file
  writer = tf.python_io.TFRecordWriter(tf_rec_filename)
  
  # iterate through all .nii files
  for meta_data in img_filenames:

    # load the image and label
    img, label = load_image_2D(meta_data, labels)

    # create a feature
    feature = {'label': _int64_feature(label),
               'image': _float_feature(img.ravel())}

    # create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))

    # serialize to string and write on the file
    writer.write(example.SerializeToString())
    
  writer.close()

In [ ]:
create_tf_record_2D(training_set, train_tfrec2D, LABELS)
create_tf_record_2D(test_set, test_tfrec2D, LABELS)
create_tf_record_2D(validation_set, val_tfrec2D, LABELS)

In [ ]:
# array where we are going to save the filenames
filenames = np.array([])

# iterate all three class folders in the supervised db
for subf in CLASS_SUBFOLDERS:
  path = DB_SS_PATH + subf
  for name in os.listdir(path):
    complete_name = os.path.join(path, name)
    if os.path.isfile(complete_name): 
      filenames = np.concatenate((filenames, complete_name), axis=None)
    
for name in os.listdir(DB_UL_SS_PATH):
  complete_name = os.path.join(DB_UL_SS_PATH, name)
  if os.path.isfile(complete_name):
      filenames = np.concatenate((filenames, complete_name), axis=None)

In [ ]:
for i in range(1000):
  np.random.shuffle(filenames)
  
test_margin = int(len(filenames) * TEST_SPLIT)
unsupervised_training_set, unsupervised_test_set = filenames[test_margin:], filenames[:test_margin]

validation_margin = int(len(unsupervised_training_set) * VALIDATION_SPLIT)
unsupervised_training_set, unsupervised_validation_set = unsupervised_training_set[validation_margin:], unsupervised_training_set[:validation_margin]
UNSUPERVISED DATA (DEPRECATED)